# Trader Performance vs Market Sentiment

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
market=pd.read_csv("fear_greed_index.csv")

In [3]:
trader=pd.read_csv("historical_data.csv")

In [4]:
market.head()

,timestamp,value,classification,date
0,1517463000,30,Fear,2018-02-01
1,1517549400,15,Extreme Fear,2018-02-02
2,1517635800,40,Fear,2018-02-03
3,1517722200,24,Extreme Fear,2018-02-04
4,1517808600,11,Extreme Fear,2018-02-05


In [5]:
trader.head()

,Account,Coin,Execution Price,Size Tokens,Size USD,Side,Timestamp IST,Start Position,Direction,Closed PnL,Transaction Hash,Order ID,Crossed,Fee,Trade ID,Timestamp
0,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9769,986.87,7872.16,BUY,02-12-2024 22:50,0.000000,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.345404,8.950000e+14,1.730000e+12
1,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9800,16.00,127.68,BUY,02-12-2024 22:50,986.524596,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.005600,4.430000e+14,1.730000e+12
2,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9855,144.09,1150.63,BUY,02-12-2024 22:50,1002.518996,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050431,6.600000e+14,1.730000e+12
3,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9874,142.98,1142.04,BUY,02-12-2024 22:50,1146.558564,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050043,1.080000e+15,1.730000e+12
4,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9894,8.73,69.75,BUY,02-12-2024 22:50,1289.488521,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.003055,1.050000e+15,1.730000e+12


# Data Cleaning

In [6]:
market.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2644 entries, 0 to 2643
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   timestamp       2644 non-null   int64 
 1   value           2644 non-null   int64 
 2   classification  2644 non-null   object
 3   date            2644 non-null   object
dtypes: int64(2), object(2)
memory usage: 82.8+ KB


In [7]:
market.isna().sum()

timestamp         0
value             0
classification    0
date              0
dtype: int64

In [8]:
market.duplicated().sum()

np.int64(0)

In [9]:
trader.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211224 entries, 0 to 211223
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Account           211224 non-null  object 
 1   Coin              211224 non-null  object 
 2   Execution Price   211224 non-null  float64
 3   Size Tokens       211224 non-null  float64
 4   Size USD          211224 non-null  float64
 5   Side              211224 non-null  object 
 6   Timestamp IST     211224 non-null  object 
 7   Start Position    211224 non-null  float64
 8   Direction         211224 non-null  object 
 9   Closed PnL        211224 non-null  float64
 10  Transaction Hash  211224 non-null  object 
 11  Order ID          211224 non-null  int64  
 12  Crossed           211224 non-null  bool   
 13  Fee               211224 non-null  float64
 14  Trade ID          211224 non-null  float64
 15  Timestamp         211224 non-null  float64
dtypes: bool(1), float64(

In [10]:
trader.isna().sum()

Account             0
Coin                0
Execution Price     0
Size Tokens         0
Size USD            0
Side                0
Timestamp IST       0
Start Position      0
Direction           0
Closed PnL          0
Transaction Hash    0
Order ID            0
Crossed             0
Fee                 0
Trade ID            0
Timestamp           0
dtype: int64

In [11]:
trader.duplicated().sum()

np.int64(0)

# Timestamps Conversion and Aligning with Date

In [12]:
market["date"] = pd.to_datetime(market["timestamp"],unit="s",errors="coerce").dt.date

In [13]:
trader["timestamp"] = pd.to_datetime(trader["Timestamp IST"], errors="coerce")
trader["date"] = trader["timestamp"].dt.date

# Key Metrics

In [14]:
fear_greed_daily = (
    market
    .groupby("date", as_index=False)
    .agg({"value": "mean"})
)

In [15]:
trader["is_win"] = trader["Closed PnL"] > 0

In [16]:
trader["leverage_proxy"] = np.where(
    trader["Start Position"].abs() > 0,
    trader["Size USD"] / trader["Start Position"].abs(),
    np.nan
)

In [17]:
daily_metrics = (
    trader
    .groupby("date", as_index=False)
    .agg(
        daily_pnl=("Closed PnL", "sum"),
        total_volume_usd=("Size USD", "sum"),
        num_trades=("Trade ID", "count"),
        win_rate=("is_win", "mean"),
        avg_trade_size=("Size USD", "mean"),
        avg_leverage=("leverage_proxy", "mean")
    )
)

daily_metrics["win_rate"] = daily_metrics["win_rate"] * 100

In [18]:
daily_pnl_per_account = (
    trader
    .groupby(["date", "Account"], as_index=False)
    .agg(daily_pnl=("Closed PnL", "sum"))
)

In [19]:
long_short = (
    trader
    .groupby(["date", "Side"])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

long_short["long_short_ratio"] = (
    long_short.get("Buy", 0) /
    long_short.get("Sell", 1)
)

In [20]:
daily_metrics = (
    daily_metrics
    .merge(long_short[["date", "long_short_ratio"]], on="date", how="left")
    .merge(fear_greed_daily, on="date", how="left")
)

In [21]:
print(daily_metrics.head())

         date    daily_pnl  total_volume_usd  num_trades   win_rate  \
0  2023-01-05     0.000000            477.00           3   0.000000   
1  2023-05-12     0.000000          50005.83           9   0.000000   
2  2024-01-01  -129.531460         264239.53          18   5.555556   
3  2024-01-02     0.000000           2008.18           6   0.000000   
4  2024-01-03  8244.241409         472974.70         137  93.430657   

   avg_trade_size   avg_leverage  long_short_ratio  value  
0      159.000000    1191.326187               0.0   29.0  
1     5556.203333  233698.062151               0.0   49.0  
2    14679.973889   41106.911531               0.0   65.0  
3      334.696667       0.009565               0.0   71.0  
4     3452.370073       0.529004               0.0   70.0  


•	**PnL & Win Rate**: Greed days yield the highest average daily PnL (~ 423) and a superior win rate (~ 59.7%). In contrast, Fear days result in average daily losses (-45) and a lower win rate (~39.5%).

•	**Trade Frequency**: Traders are twice as active during Greed phases, averaging 29.6 trades per day, compared to only 14.4 trades during Fear periods. This indicates that high sentiment drives engagement and volume.

•	**High vs. Low Leverage**: Traders/Days using Low Leverage outperformed significantly, with an average PnL of +349, whereas High Leverage days averaged a slight loss (-5.27). This confirms that excessive leverage often wipes out the gains made during trending markets.